# Projeto 1 - Ciência dos Dados

### Import das bibliotecas

In [185]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [186]:
print(os.getcwd())

/Users/gustavopinheiro


### Leitura dos dados:

In [187]:
# Fazer caminho para o arquivo
dados = pd.read_excel('Projeto_1_CDados.xlsx')

In [188]:
dados.shape

(750, 3)

### Transformando em variáveis qualitativas:

In [189]:
dados['Classificação'] = dados['Classificação'].astype('category')
dados.Classificação.cat.categories = ['Irrelevante', 'Neutro', 'Relevante']
dados['Finalidade'] = dados['Finalidade'].astype('category')
dados.Finalidade.cat.categories = ['Teste', 'Treinamento']

In [190]:
dados.head(10)
dados.tail(10)

,Tweets,Classificação,Finalidade
740,"Cara o IPhone 11 tá muito feio, que designe é ...",Relevante,Treinamento
741,QUE DESIGNE HORRÍVEL! iPhone 11,Relevante,Treinamento
742,odiei o designe do iPhone 11 porém queria mt m...,Relevante,Treinamento
743,"o designe do iphone 11 ta horrível, era melhor...",Relevante,Treinamento
744,"Apple e seu designe incrível, ai chegou iPhone...",Relevante,Treinamento
745,"Vey namoral esse iPhone 11 ficou muito feio , ...",Relevante,Treinamento
746,Se realmente o novo iPhone 11 for esse que est...,Relevante,Treinamento
747,O iPhone 11 o designe não é o mais bonito mas ...,Neutro,Treinamento
748,Steve Jobs vai ressuscitar e dar uma paulada e...,Irrelevante,Treinamento
749,Pior de tudo que já tem empresas chinesas copi...,Neutro,Treinamento


In [191]:
dados['Tweets']

0      quem tem iphone 11 ta notando que a saúde da b...
1      A mulher foi embora e deixou o iPhone 11 dela ...
2      Se for ver bem a gente consegue trocar um ipho...
3      A câmera não é a do iPhone 8. A frontal quase ...
4      uma hora eu procuro comunidades hippies no bra...
                             ...                        
745    Vey namoral esse iPhone 11 ficou muito feio , ...
746    Se realmente o novo iPhone 11 for esse que est...
747    O iPhone 11 o designe não é o mais bonito mas ...
748    Steve Jobs vai ressuscitar e dar uma paulada e...
749    Pior de tudo que já tem empresas chinesas copi...
Name: Tweets, Length: 750, dtype: object

### Definindo variáveis de treinamento e de teste:

In [ ]:
train = dados.loc[dados['Finalidade'] == 'Treinamento',:]
test = dados.loc[dados['Finalidade'] == 'Teste',:]

In [193]:
train.shape

(501, 3)

In [194]:
train.tail()

,Tweets,Classificação,Finalidade
745,"Vey namoral esse iPhone 11 ficou muito feio , ...",Relevante,Treinamento
746,Se realmente o novo iPhone 11 for esse que est...,Relevante,Treinamento
747,O iPhone 11 o designe não é o mais bonito mas ...,Neutro,Treinamento
748,Steve Jobs vai ressuscitar e dar uma paulada e...,Irrelevante,Treinamento
749,Pior de tudo que já tem empresas chinesas copi...,Neutro,Treinamento


In [195]:
test.shape

(249, 3)

In [196]:
test.head()

,Tweets,Classificação,Finalidade
0,quem tem iphone 11 ta notando que a saúde da b...,Relevante,Teste
1,A mulher foi embora e deixou o iPhone 11 dela ...,Irrelevante,Teste
2,Se for ver bem a gente consegue trocar um ipho...,Relevante,Teste
3,A câmera não é a do iPhone 8. A frontal quase ...,Relevante,Teste
4,uma hora eu procuro comunidades hippies no bra...,Relevante,Teste


### Probabilidade de um tweet do total de tweets ser irrelevante, neutro ou relevante:

In [207]:
prob_clas = dados.Classificação.value_counts(True, sort=False)
P_I = prob_clas[0]
P_N = prob_clas[1]
P_R = prob_clas[2]

print(f'Probabilidade de ser irrelevante: {P_I:0.5g}')
print(f'Probabilidade de ser neutra: {P_N:0.5g}')
print(f'Probabilidade de ser relevante: {P_R:0.5g}')

Probabilidade de ser irrelevante: 0.25768
Probabilidade de ser neutra: 0.37517
Probabilidade de ser relevante: 0.36716


### Probabilidade de um tweet do total de tweets ser de treinamento ou de teste:

In [209]:
prob_fina = dados.Finalidade.value_counts(True, sort=False)
P_trei = prob_fina[1]
P_test = prob_fina[0]

print(f'Probabilidade de ser de treinamento: {P_trei:0.5g}')
print(f'Probabilidade de ser de teste: {P_test:0.5g}')

Probabilidade de ser de treinamento: 0.668
Probabilidade de ser de teste: 0.332


3. Montando o classificador Naive-Bayes:
Considerando apenas as mensagens da planilha Treinamento, o objetivo aqui é ensinar
o seu classificador quais são as palavras mais comuns (frequentes) em uma mensagem
relevante e as mais presentes nas mensagens irrelevantes
Nesse caso, seu código deve conter:
✓ Limpeza de mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não
remover emojis. #Ok
✓ Correção de espaços entre palavras e/ou emojis. 
✓ Proposta 

### Definição da função de "limpeza" dos tweets

In [199]:

#Para remover pontuação, transformar em letras minúsculas e 
def cleanup(text):
    punctuation = r'[~^´``()\/|"''˜%ˆ&#@*;:<>{}=,!-.:?;]'
    pattern = re.compile(punctuation) # Tira a pontuação
    text = re.sub(pattern, "", text)
    text = re.sub("https", "", text)
    text = re.sub("\n", "", text)
    text = text.lower()
    text = text.split()
    #text = " ".join(text.split()) # Separa as palavras
    return text

### Separando por classificação e limpando os tweets de treinamento:

In [200]:
# Separando os tweets de treinamento pela classificação:

tweets_irel = train.loc[train['Classificação'] == 'Irrelevante', :]
tweets_neut = train.loc[train['Classificação'] == 'Neutro', :]
tweets_rel = train.loc[train['Classificação'] == 'Relevante', :]


# Limpando os tweets treinamento:

tweets_irel_limpos = tweets_irel['Tweets'].apply(cleanup)
tweets_neut_limpos = tweets_neut['Tweets'].apply(cleanup)
tweets_rel_limpos = tweets_rel['Tweets'].apply(cleanup)

print(tweets_irel_limpos.tail(20))


668    [alguém, me, da, um, iphone, 11, amarelo, eu, ...
678    [eles, usam, remini, ou, iphone, 11, pro, max,...
680    [incrível, a, capacidade, da, anahi, ter, um, ...
681    [puts, mal, sabe, ele, que, o, xiaomi, mi, 10,...
684    [alguém, sabe, me, dizer, pq, o, iphone, 11, p...
691    [a, câmera, do, samsung, deles, é, melhor, dq,...
693    [pra, vc, ver, que, até, no, celular, ele, eco...
695    [eu, so, queria, um, iphone, 11, ou, um, samsu...
699    [viajando, do, lado, de, uma, poc, riquíssima,...
700    [minha, mãe, querendo, comprar, o, iphone, 11,...
702    [pai, n, aguenta, mais, samsung, vou, ter, q, ...
703    [e, meu, pai, que, saiu, pra, comprar, um, sam...
705    [vocês, tem, noção, que, o, celular, mais, ven...
708    [kkkkkkk, comprou, o, galaxy, note, 20, ultra,...
715    [fotos, a, direita, são, do, iphone, 11, e, a,...
726    [o, iphone, 11, é, igual, o, iphone, xr, com, ...
727    [só, um, iphone, 11, conseguiria, me, empodera...
728    [alguém, me, explica, pq

### Juntando as listas formadas por cada tweet em apenas uma lista, que contém todas as palavras presentes em uma determinada classificação de tweets:

In [215]:
# Juntar as listas como uma só lista de palavras

todas_palavras_irel = []
todas_palavras_neut = []
todas_palavras_rel = []

for i in tweets_irel_limpos:
    for p in i:
        todas_palavras_irel.append(p)
        
for i in tweets_neut_limpos:
    for p in i:
        todas_palavras_neut.append(w)
        
for i in tweets_rel_limpos:
    for p in i:
        todas_palavras_rel.append(w)
        
todas_palavras_irel


['vendo',
 'saco',
 'de',
 'arroz',
 'tio',
 'joão',
 'seminovo',
 'ou',
 'troco',
 'por',
 'iphone',
 '11',
 'a',
 'pessoa',
 'tem',
 'um',
 'iphone',
 '11',
 'e',
 'tira',
 'umas',
 'foto',
 'com',
 'qualidade',
 'mais',
 'duvidosa',
 'q',
 'meu',
 'motorola',
 'comprei',
 'um',
 'iphone',
 '11',
 'demorei',
 'meio',
 'século',
 'a',
 'adaptarme',
 'agora',
 'que',
 'sei',
 'onde',
 'estão',
 'as',
 'coisas',
 'todas',
 'querem',
 'lançar',
 'uma',
 'atualização',
 'que',
 'vai',
 'mudar',
 'tudo',
 'de',
 'lugar',
 'e',
 'vocês',
 'acham',
 'que',
 'isso',
 'é',
 'uma',
 'coisa',
 'boa',
 'seus',
 'geeks',
 'sem',
 'sentimentos',
 'não',
 'consegui',
 'comprar',
 'nem',
 'o',
 'iphone',
 '11',
 'já',
 'vai',
 'ter',
 'o',
 '12',
 'burrice',
 'pra',
 'mim',
 'é',
 'o',
 'ser',
 'humano',
 'comprar',
 'um',
 'iphone',
 '11',
 'e',
 'preferir',
 'ficar',
 'com',
 'o',
 '6s',
 'porque',
 'o',
 'celular',
 'não',
 'fez',
 'backup',
 'das',
 'conversas',
 'do',
 'wpp',
 'tenho',
 'um',
 '

### Transformando a lista com as palavras em uma série:

In [225]:
serie_palavras_irel = pd.Series(todas_palavras_irel)
serie_palavras_neut = pd.Series(todas_palavras_neut)
serie_palavras_rel = pd.Series(todas_palavras_rel)

serie_palavras_irel


0          vendo
1           saco
2             de
3          arroz
4            tio
          ...   
2246     designe
2247    horrível
2248          do
2249      iphone
2250          11
Length: 2251, dtype: object

### Fazendo a contagem do número de ocorrências de uma palavra dentro da série (Frequência absoluta):

In [219]:
tabela_palavras_irel = serie_palavras_irel.value_counts()
tabela_palavras_neut = serie_palavras_neut.value_counts()
tabela_palavras_rel = serie_palavras_rel.value_counts()

tabela_palavras_irel

iphone       125
11           104
o             87
que           66
a             56
            ... 
mão            1
odioooo        1
aparelhos      1
nesta          1
outro          1
Length: 746, dtype: int64

### Fazendo a Frequência Relativa:

In [222]:
tabela_rel_palavras_irel = serie_palavras_irel.value_counts(True)
tabela_rel_palavras_neut = serie_palavras_neut.value_counts(True)
tabela_rel_palavras_rel = serie_palavras_rel.value_counts(True)

tabela_rel_palavras_irel

iphone       0.055531
11           0.046202
o            0.038649
que          0.029320
a            0.024878
               ...   
mão          0.000444
odioooo      0.000444
aparelhos    0.000444
nesta        0.000444
outro        0.000444
Length: 746, dtype: float64

4. Verificando a performance:
Considerando agora apenas as mensagens da planilha Teste, seu objetivo aqui é testar a
qualidade do seu classificador.
Para tanto, você deve extrair as seguintes contagens:
✓ Porcentagem de verdadeiros positivos (mensagens relevantes e que são
classificadas como relevantes)
✓ Porcentagem de falsos positivos (mensagens irrelevantes e que são classificadas
como relevantes)
✓ Porcentagem de verdadeiros negativos (mensagens irrelevantes e que são
classificadas como irrelevantes)
✓ Porcentagem de falsos negativos (mensagens relevantes e que são classificadas
como irrelevantes)
Opcionalmente:
✓ Criar categorias intermediárias de relevância baseado na diferença de
probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito
irrelevante.

In [231]:
# Probabilidade de ser relevante dado que é relevante

# Probabilidade de não ser relevante dado que é relevante

# Probabilidade de ser relevante dado que não é relevante

# Probabilidade de ser neutra dado que é neutra

# Probabilidade de ser neutra dado que não é neutra

#Probabilidade de ser irrelevante dado que é irrelevante

# Probabilidade de ser irrelevante dado que não é irrelevante
probtweetdadorel = 1
for w in tweets_test:
    
        
probtweetdadorel *= tabela_rel_palavras_rel[i]

KeyError: 'quem'

In [ ]:
prob_irel = len(serie_palavras_irel)
prob_neut = len(serie_palavras_neut)
prob_rel = len(serie_palavras_rel)
print (prob

In [229]:
# Separando os tweets de teste pela classificação:
#tweets_irel2 = test.loc[test['Classificação'] == 'Irrelevante', :]
#tweets_neut2 = test.loc[test['Classificação'] == 'Neutro', :]
#tweets_rel2 = test.loc[test['Classificação'] == 'Relevante', :]

#tweets_test = tweets_irel2 + tweets_neut2 + tweets_rel2

tweets_test = test['Tweets'].apply(cleanup)

tweets_test

0      [quem, tem, iphone, 11, ta, notando, que, a, s...
1      [a, mulher, foi, embora, e, deixou, o, iphone,...
2      [se, for, ver, bem, a, gente, consegue, trocar...
3      [a, câmera, não, é, a, do, iphone, 8, a, front...
4      [uma, hora, eu, procuro, comunidades, hippies,...
                             ...                        
244    [iphone, 11, lançou, ontem, e, já, vi, uns, 3,...
245    [ontade, de, lançar, um, iphone, 11, no, começ...
246    [gente, quero, o, novo, iphone, 11, na, minha,...
247    [concordo, mas, se, quiser, me, dar, o, iphone...
248    [não, vejo, a, hora, de, o, iphone, 11, chegar...
Name: Tweets, Length: 249, dtype: object

In [ ]:
todas_palavras_test = []
for i in 

5. Concluindo:
Faça um comparativo qualitativo sobre os percentuais obtidos para que possa discutir a
performance do seu classificador.
Explique como são tratadas as mensagens com dupla negação e sarcasmo.
Proponha um plano de expansão. Por que eles devem continuar financiando o seu
projeto?
Opcionalmente:
✓ Discorrer por que não posso alimentar minha base de Treinamento
automaticamente usando o próprio classificador, aplicado a novos tweets.
✓ Propor diferentes cenários de uso para o classificador Naive-Bayes. Pense em
outros cenários sem intersecção com este projeto.
✓ Sugerir e explicar melhorias reais no classificador com indicações concretas de
como implementar (não é preciso codificar, mas indicar como fazer. Indique
material de pesquisa sobre o assunto).